In [102]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

## HERA - Individual Performance

In [103]:
hera_perf = pd.read_csv('hera_ind_perf.csv').dropna()
hera_perf_date = list(hera_perf.Date.unique())

hera_smm1 = pd.read_csv('hera_smm1_survey.csv').drop(columns = ['Campaign', 'Mission', 'Session'])
hera_smm1_date = list(hera_smm1.StartDate.unique())

hera_smm2 = pd.read_csv('hera_smm2_survey.csv').drop(columns = ['Campaign', 'Mission', 'Session'])
hera_smm2_date = list(hera_smm2.StartDate.unique())

mutual_date = list(set(hera_perf_date).intersection(set(hera_smm1_date)).intersection(set(hera_smm2_date)))

hera_perf = hera_perf.loc[hera_perf.Date.isin(mutual_date), :]

role_map = {'Drill':1, 'Mats':2, 'Ops':3, 'Bio':4, 'Fluid':5, 'Mech':6, 'Hydro':7, 'Sedi':8,
           'Struct':9, 'Terrain':10, 'Maint':11, 'Meteor':12}

hera_perf['role'] = hera_perf['RED_Role'].map(role_map)
hera_perf = hera_perf.loc[:, ['Date', 'IndPerformance', 'role']]
hera_perf['date'] = pd.to_datetime(hera_perf['Date'])
hera_perf = hera_perf.sort_values(by = ['date', 'role']).loc[:, ['Date', 'role', 'IndPerformance']]

## HERA - SMM1

In [120]:
hera = hera_smm1.loc[hera_smm1['StartDate'].isin(mutual_date), :]
mission = hera.StartDate.unique()

smm1_perf = hera.loc[:, ['StartDate', 'Q2.1']].rename(columns = {'StartDate':'Date', 'Q2.1':'role'}).merge(
    hera_perf, on = ['Date', 'role'])

smm1_perf.loc[:, ['IndPerformance']].rename(columns = {'IndPerformance':'smm1_perf'}).to_csv('smm1_perf.csv')

In [112]:
hera_smm = []

for i in mission:
    m = hera[hera['StartDate'] == i].iloc[:, 2:].to_numpy()
    row, col = m.shape
    smm = []
    for j in range(row - 1):
        for k in list(range(j + 1, row)):
            temp = m[j] - m[k]
            smm.append([j + 1, k + 1, np.sqrt(np.dot(temp.T, temp)) / 10 * np.sqrt(10)])
    hera_smm.append((i, smm))

In [113]:
hera_link = {}

for i in range(len(hera_smm)):
    m = hera_smm[i][1]
    smm = [x[2] for x in m]
    avg_smm = sum(smm) / len(smm)
    link = [x[:2] for x in m if x[2] < avg_smm]
    hera_link[[x[0] for x in hera_smm][i]] = link 

In [114]:
hera_date = list(hera_link.keys())
adjacency_list = []
for i in hera_date:
    edge_list = hera_link[i]
    size = hera[hera['StartDate'] == i].shape[0]
    adjacency = [[0]*size for _ in range(size)]
    for sink, source in edge_list:
        adjacency[sink - 1][source - 1] = 1
    for i in range(size):
        for j in range(size):
            t = adjacency[i][j]
            adjacency[j][i] = t
    adjacency_list.append(adjacency)

In [115]:
mat_list = []
for i in range(len(adjacency_list)):
    mat = adjacency_list[i]
    front_sum = 0
    back_sum = 0
    for j in range(i):
        prev_mat = adjacency_list[j]
        front_sum = front_sum + np.array(prev_mat).shape[0]
    for k in range(i + 1, len(adjacency_list)):
        post_mat = adjacency_list[k]
        back_sum = back_sum + np.array(post_mat).shape[0]
    mat_list.append(np.block([np.zeros((np.array(mat).shape[0], front_sum)), 
                              np.array(mat), np.zeros((np.array(mat).shape[0], back_sum))]))

In [117]:
concat_mat = mat_list[0]
for i in list(range(1, len(mission))):
    concat_mat = np.concatenate([concat_mat, mat_list[i]])

In [118]:
block_diag_mat = pd.DataFrame(concat_mat).astype('int')

In [119]:
print(block_diag_mat.shape)
block_diag_mat.iloc[:20, :20]

(289, 289)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0
1,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0
3,1,0,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,1,1,1,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,1,1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0


In [123]:
block_diag_mat.to_csv('smm1_block_diag_mat.csv')

## HERA - SMM2

In [124]:
hera = hera_smm2.loc[hera_smm2['StartDate'].isin(mutual_date), :]
mission = hera.StartDate.unique()

smm2_perf = hera.loc[:, ['StartDate', 'Q2.1']].rename(columns = {'StartDate':'Date', 'Q2.1':'role'}).merge(
    hera_perf, on = ['Date', 'role'])

smm2_perf.loc[:, ['IndPerformance']].rename(columns = {'IndPerformance':'smm2_perf'}).to_csv('smm2_perf.csv')

In [126]:
hera_smm = []

for i in mission:
    m = hera[hera['StartDate'] == i].iloc[:, 2:].to_numpy()
    row, col = m.shape
    smm = []
    for j in range(row - 1):
        for k in list(range(j + 1, row)):
            temp = m[j] - m[k]
            smm.append([j + 1, k + 1, np.sqrt(np.dot(temp.T, temp)) / 10 * np.sqrt(10)])
    hera_smm.append((i, smm))

In [127]:
hera_link = {}

for i in range(len(hera_smm)):
    m = hera_smm[i][1]
    smm = [x[2] for x in m]
    avg_smm = sum(smm) / len(smm)
    link = [x[:2] for x in m if x[2] < avg_smm]
    hera_link[[x[0] for x in hera_smm][i]] = link 

In [128]:
hera_date = list(hera_link.keys())
adjacency_list = []
for i in hera_date:
    edge_list = hera_link[i]
    size = hera[hera['StartDate'] == i].shape[0]
    adjacency = [[0]*size for _ in range(size)]
    for sink, source in edge_list:
        adjacency[sink - 1][source - 1] = 1
    for i in range(size):
        for j in range(size):
            t = adjacency[i][j]
            adjacency[j][i] = t
    adjacency_list.append(adjacency)

In [129]:
mat_list = []
for i in range(len(adjacency_list)):
    mat = adjacency_list[i]
    front_sum = 0
    back_sum = 0
    for j in range(i):
        prev_mat = adjacency_list[j]
        front_sum = front_sum + np.array(prev_mat).shape[0]
    for k in range(i + 1, len(adjacency_list)):
        post_mat = adjacency_list[k]
        back_sum = back_sum + np.array(post_mat).shape[0]
    mat_list.append(np.block([np.zeros((np.array(mat).shape[0], front_sum)), 
                              np.array(mat), np.zeros((np.array(mat).shape[0], back_sum))]))

In [130]:
concat_mat = mat_list[0]
for i in list(range(1, len(mission))):
    concat_mat = np.concatenate([concat_mat, mat_list[i]])

In [134]:
block_diag_mat = pd.DataFrame(concat_mat).astype('int')
print(block_diag_mat.shape)
block_diag_mat.iloc[:20, :20]

(288, 288)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
2,1,1,0,1,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,1,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0
7,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
8,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
9,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
